In [ ]:
!pip install dateparser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 KB 4.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re, datetime
import dateparser

In [ ]:
# datanı oxuma
data=pd.read_csv('/content/Data (1).csv',header=None)
data

,0
0,3 PM Mon 24th-Mar-2014___0.384 kwh
1,5AM 15-Aug-2014___1.201 kwh
2,__8PM Thu 20-Mar-2014____1.523 kwh
3,6PM 23rd-Apr-2014___0.424 kwh
4,_1AM Friday 19th-Dec-2014___0.209 kwh
...,...
8755,_1AM Fri 07th-Nov-2014_0.084 kwh
8756,_6AM 20-May-2014__1.027 kwh
8757,__2 AM Tuesday 8th-Apr-2014___0.052 kwh
8758,9 PM 27th-Jan-2014_1.428 kwh


In [ ]:
# tarixlər və saatlar olacaq boş panellər
dates = []
hours = []


In [ ]:
# xam datadan tarixləri və saatları çıxarma
for i in range(len(data)):
  string = str(data.iloc[i][0])
  nums = re.findall(r'\d+', string)
  if nums[0]==nums[1]:
    string=string[string.index(nums[1])+1:]
    pattern = str(nums[1])+'(.*?)'+str(nums[2])
    parsed_date = re.search(pattern, string).group(0)
    dates.append(parsed_date)
    hours.append(nums[0])
  else:
    pattern = str(nums[1])+'(.*?)'+str(nums[2])
    parsed_date = re.search(pattern, string).group(0)
    dates.append(parsed_date)
    hours.append(nums[0])

In [ ]:
# tarix və saat panelini əlavə olunması
data['dates']=dates
data['hours']=hours


In [ ]:
# kilovat/saat panelinin boş massivi
KwHs=[]
# kilovat/saat saylarının doldurulması
for n in range(len(data)):
  pattern = r'2014(.*?)kwh'
  parsed_date = re.search(pattern, str(data.iloc[n][0])).group(1)
  KwHs.append(float(re.findall(r'\d+', parsed_date)[0]+'.'+re.findall(r'\d+', parsed_date)[1]))


In [ ]:
# kilovat/saat panelinin yaradılması
data['KwHs']=KwHs


In [ ]:
# tarixlərin yeni formata keçirilməsi
dates = []
for i in range(len(data['dates'])):
  dates.append(dateparser.parse(data['dates'][i]))

data['dates']=dates

# birinci sualın cavabı
data['KwHs'].mean()


0.7818779680365298

In [ ]:
# tarixlərdən aylar götürülərək yeni xananın yaradılması
data['month'] = data['dates'].dt.month

# ikinci sualın cavabı
data[data['month']==2]['KwHs'].mean()

0.8329151785714286

In [ ]:
# tarixlərdən həftənin günləri xanasının yaradılması

data['days_of_week'] = data['dates'].dt.day_name()

# üçüncü sualın cavabı

for i in data['days_of_week'].unique():
  print(i, '--',data[data['days_of_week']==i]['KwHs'].mean())


Monday -- 0.7259447115384615
Friday -- 0.7537443910256411
Thursday -- 0.7454575320512821
Wednesday -- 0.7339426100628931
Tuesday -- 0.7260576923076923
Saturday -- 0.890763621794872
Sunday -- 0.8981570512820511


In [ ]:
# saatların AM və ya PM olmasını təyin etmə
am_pm = ['PM'if len(re.findall('PM', data.iloc[i][0]))==1 else 'AM' for i in range(len(data))]
data['AM\PM'] = am_pm

In [ ]:
# AM və ya PM sütununun yaradılması
data['hours'] = data['hours']+':00 '+ data['AM\PM']

In [ ]:
# AM və ya PM formatında asılı olaraq saatların 24 saatlıq formata keçirilməsi
data['hours'] = pd.to_datetime(data['hours']).dt.strftime('%H:%M:%S')

In [ ]:
# tarixlərin yenidən təmizlənməsi
data['dates'] = data['dates'].dt.date

In [ ]:
# 4 saatlıq fasiləsiz iş rejimində istifadə olunan maksimum enerji miqdarı
max = 0
for i in range(len(data)):
  if max < data.sort_values(by=['dates', 'hours'])['KwHs'].iloc[i:i+4].sum():
    max = data.sort_values(by=['dates', 'hours'])['KwHs'].iloc[i:i+4].sum()
print(max)

17.237000000000002


In [ ]:
data

,0,dates,hours,KwHs,month,days_of_week,AM\PM
0,3 PM Mon 24th-Mar-2014___0.384 kwh,2014-03-24,15:00:00,0.384,3,Monday,PM
1,5AM 15-Aug-2014___1.201 kwh,2014-08-15,05:00:00,1.201,8,Friday,AM
2,__8PM Thu 20-Mar-2014____1.523 kwh,2014-03-20,20:00:00,1.523,3,Thursday,PM
3,6PM 23rd-Apr-2014___0.424 kwh,2014-04-23,18:00:00,0.424,4,Wednesday,PM
4,_1AM Friday 19th-Dec-2014___0.209 kwh,2014-12-19,01:00:00,0.209,12,Friday,AM
...,...,...,...,...,...,...,...
8755,_1AM Fri 07th-Nov-2014_0.084 kwh,2014-11-07,01:00:00,0.084,11,Friday,AM
8756,_6AM 20-May-2014__1.027 kwh,2014-05-20,06:00:00,1.027,5,Tuesday,AM
8757,__2 AM Tuesday 8th-Apr-2014___0.052 kwh,2014-04-08,02:00:00,0.052,4,Tuesday,AM
8758,9 PM 27th-Jan-2014_1.428 kwh,2014-01-27,21:00:00,1.428,1,Monday,PM


In [ ]:
data['dates'] = pd.to_datetime(data['dates'])

data.to_excel('AISCO_cleaned_data.xlsx')
